# Introduction

Let's have a global view on the train set.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
from skimage.transform import resize
import rasterio

import cv2
def resize(img):
    imgt = img.transpose(1,2,0)
    new_img = np.zeros((500,500,3)).astype(np.uint8)
    for i in range(3):
        temp = imgt[:,:,i]
        new_img[:,:,i] = cv2.resize(temp,(500,500))
    return new_img, np.array(temp.shape)/500

import json
from rasterio.mask import mask

def gen_masks(path, raster):
    with open(path, 'r') as f:
        aois_1 = json.load(f)
    
    all_aois_1 = [elmt['geometry'] for elmt in aois_1]
    mask1, transform = mask(raster, all_aois_1, crop = False)
    mask1, _ = resize(mask1)
    return mask1

# Get train ids

In [ ]:
ids = glob.glob('/kaggle/input/hubmap-kidney-segmentation/train/*.tiff')
ids = [elmt.split('/')[-1].split('.')[0] for elmt in ids]
ids

# Create low resolution images and masks

In [ ]:
from tqdm import tqdm
imgs = []
masks1 = []
masks2 = []

for id_ in ids:
    
    path_tiff = f'/kaggle/input/hubmap-kidney-segmentation/train/{id_}.tiff'
    path_json1 = f'/kaggle/input/hubmap-kidney-segmentation/train/{id_}-anatomical-structure.json'
    path_json2 = f'/kaggle/input/hubmap-kidney-segmentation/train/{id_}.json'

    raster = rasterio.open(path_tiff)
    img = raster.read()
    crs = raster.crs
    transform = raster.transform

    
    nimg, trans = resize(img)
    ts = nimg.transpose(2,1,0)
    transform2 = transform * transform.scale(trans[1],trans[0])
    with rasterio.open('temp.tif','w',driver='GTiff',
           height=ts.shape[1],
           width=ts.shape[2],
           count=ts.shape[0],
           dtype=ts.dtype,
           crs = crs,
           transform=transform2,) as f:
        
        f.write(ts)

    subraster = rasterio.open('temp.tif')
    mask1 = gen_masks(path_json1, subraster)
    mask2 = gen_masks(path_json2, subraster)
    mask1[mask1 >0] = 255
    mask2[mask2 >0] = 255
    imgs.append(nimg)
    masks1.append(mask1)
    masks2.append(mask2)

In [ ]:
plt.imshow(nimg)
plt.imshow(mask1, alpha = 0.5)
plt.imshow(mask2, alpha = 0.5)

# Assemble everything

In [ ]:
big_picture = np.zeros((500*2,500*4,3)).astype(np.uint8)
big_mask1 = np.zeros((500*2,500*4,3)).astype(np.uint8)
big_mask2 = np.zeros((500*2,500*4,3)).astype(np.uint8)
k = 0
for i in range(2):
    for j in range(4):
        big_picture[500*i:500*(i+1),500*j:500*(j+1),:] = imgs[k]
        big_mask1[500*i:500*(i+1),500*j:500*(j+1),:] = masks1[k]
        big_mask2[500*i:500*(i+1),500*j:500*(j+1),:] = masks2[k]
        k+=1

# Visualisation

In [ ]:
plt.figure(figsize = (30,30))
plt.title('All Images and masks', size = 30)
fig = plt.imshow(big_picture)
plt.imshow(big_mask1, alpha = 0.5)
plt.imshow(big_mask2, alpha = 0.5)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)
plt.show()